Вариант с knn на чеках и NN на только характеристиках

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('corr_sales_biv_preprocess_full.csv', encoding='cp1251', sep=',', low_memory=False)
pd.set_option('display.max_columns', None)
df.head(5)
df.drop('Unnamed: 0', axis= 1 , inplace= True )

In [3]:
#df.info(verbose = True, show_counts = True)

In [4]:
df2_full = df.iloc[:,[1,16,27,28,32,33,34,35,40,43,44]]
df2_data = df.iloc[:,[16,27,28,32,33,34,35,40,43,44]]
df2_data.head(5)

,r_value,height,inner_tent_height,depth,rope_diameter,sl_bag_length,sl_bag_height,main_axe_load,volume,real_weight
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.709
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.709
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.709
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.709
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.709


In [5]:
df3 = df.iloc[:,55:483]
df4 = df.iloc[:,491:529]
df_fin = df.iloc[:,487:489]
#df3.head(5)

In [6]:
# Вариант с данными в нейронку только по характеристикам товара и чеки отдельно только в KNN
df_toteach = pd.concat([df2_data,df3], axis=1)
df_tocheck = pd.concat([df2_full,df3], axis=1)

df_toteach = pd.concat([df_toteach,df4], axis=1)
df_tocheck = pd.concat([df_tocheck,df4], axis=1)

df_toteach = pd.concat([df_toteach,df_fin], axis=1)
df_tocheck = pd.concat([df_tocheck,df_fin], axis=1)
#df_toteach.info(verbose = True, show_counts = True)

In [7]:
df_nano_full0 = df.iloc[:,[1,12,13, 14]]
df_nano_data0 = df.iloc[:,[12,13, 14]]
df_nano_full0.head(5)

,name,discount_sum,total_sum,prod_unit_price
0,Тент ENO ProFly XL Sil Lichen,2580.0,10320.0,12900.0
1,Тент ENO ProFly XL Sil Lichen,2580.0,10320.0,12900.0
2,Тент ENO ProFly XL Sil Lichen,2580.0,10320.0,12900.0
3,Тент ENO ProFly XL Sil Lichen,6120.0,6780.0,12900.0
4,Тент ENO ProFly XL Sil Lichen,1935.0,10965.0,12900.0


In [8]:
df_nano2 = df.iloc[:,483:487]
df_nano3 = df.iloc[:,489:491]
df_nano3.head(5)

,customer_keys_le,uid_docs_le
0,1927,4248
1,880,2501
2,1273,4021
3,2581,5537
4,2056,6523


In [9]:
df_nano_teach = pd.concat([df_nano_data0,df_nano2], axis=1)
df_nano_teach =  pd.concat([df_nano_teach,df_nano3], axis=1)
#df_nano_teach.head(5)
df_nano_tocheck = pd.concat([df_nano_full0,df_nano2], axis=1)
df_nano_tocheck = pd.concat([df_nano_tocheck,df_nano3], axis=1)
df_nano_tocheck.head(5)

,name,discount_sum,total_sum,prod_unit_price,quarter_1,quarter_2,quarter_3,quarter_4,customer_keys_le,uid_docs_le
0,Тент ENO ProFly XL Sil Lichen,2580.0,10320.0,12900.0,0,1,0,0,1927,4248
1,Тент ENO ProFly XL Sil Lichen,2580.0,10320.0,12900.0,0,1,0,0,880,2501
2,Тент ENO ProFly XL Sil Lichen,2580.0,10320.0,12900.0,0,1,0,0,1273,4021
3,Тент ENO ProFly XL Sil Lichen,6120.0,6780.0,12900.0,0,0,1,0,2581,5537
4,Тент ENO ProFly XL Sil Lichen,1935.0,10965.0,12900.0,0,0,1,0,2056,6523


In [10]:
#df_toteach.info(verbose = True, show_counts = True)

In [11]:
X = df_nano_teach.iloc[:,:-1]
y = df_nano_teach.iloc[:,-1:].to_numpy().ravel()

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [13]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors = 600)

In [14]:
clf.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=600)

In [15]:
print("Прогноз на тестовом наборе: {}".format(clf.predict(X_test)))

Прогноз на тестовом наборе: [ 291 4113  687 ... 4113 5488 2288]


In [16]:
print ("Правильность на тестовом наборе: {}".format(clf.score(X_test, y_test)))

Правильность на тестовом наборе: 0.011710581346716854


In [17]:
uid_dc_labels = clf.predict(X)

In [18]:
df_nano_tocheck['clusters'] = uid_dc_labels

#df_nano_tocheck.head(5)

In [19]:
df_inside_cluster = df_nano_tocheck.loc[(df_nano_tocheck['clusters'] == 1131)]
df_inside_cluster.head(20)

,name,discount_sum,total_sum,prod_unit_price,quarter_1,quarter_2,quarter_3,quarter_4,customer_keys_le,uid_docs_le,clusters
94,Ботинки женские Hanwag Tatra II Bunion Lady GT...,6656.0,26624.0,33280.0,0,0,1,0,3621,1323,1131
96,Ботинки женские Hanwag Tatra II Bunion Lady GT...,4992.0,28288.0,33280.0,0,0,0,1,184,560,1131
99,Ботинки женские Hanwag Tatra II Bunion Lady GT...,3328.0,29952.0,33280.0,0,0,1,0,1663,3476,1131
100,Ботинки женские Hanwag Tatra II Bunion Lady GT...,3571.0,29709.0,33280.0,0,0,1,0,5198,2358,1131
154,Несессер Peak Performance Vertical Toiletry Black,375.0,7125.0,7500.0,1,0,0,0,2769,3202,1131
166,Шорты женские Krimson Klover Rory 7 Forest,1719.0,6871.0,8590.0,0,1,0,0,3105,2438,1131
177,Палатка Alexika Tower 3 Plus Green,6840.0,27360.0,34200.0,0,1,0,0,4474,5914,1131
207,Куртка женская Ternua Vilma Hood Down Fresh Ocean,4799.0,27191.0,31990.0,0,0,0,1,2041,3947,1131
308,Бандана Buff Windproof Xido Grey,758.0,3032.0,3790.0,1,0,0,0,3011,2991,1131
373,Покрывало Matador Pocket Blanket 3.0 с зеленым...,1035.0,2415.0,3450.0,0,0,1,0,2799,1131,1131


In [21]:
#df_nano_tocheck.clusters.info(verbose = True, show_counts = True)

In [22]:
#df_nano_tocheck.clusters.value_counts(dropna=False).head(50)

In [23]:
#df_nano_tocheck.clusters.unique()

In [24]:
df_nano_tocheck.clusters.nunique()

356

In [25]:
df_nano_tocheck.uid_docs_le.nunique()

7290

In [26]:
df_toteach['clusters'] = uid_dc_labels
df_tocheck['clusters'] = uid_dc_labels
#df_tocheck.head(5)

In [27]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [28]:
le.fit(df_toteach['clusters'])
clusters_kne = le.transform(df_toteach['clusters'])
df_toteach = df_toteach.copy()
df_toteach['clusters'] = clusters_kne

df_tocheck['clusters'] = clusters_kne

In [29]:
df_inside_cluster = df_tocheck.loc[(df_tocheck['clusters'] == 7)]
#df_inside_cluster.head(20)

In [30]:
#df_tocheck.clusters.value_counts(dropna=False).head(50)

In [31]:
df_tocheck.clusters.nunique()

356

In [32]:
#df_toteach.info(verbose = True, show_counts = True)

In [33]:
data = df_toteach.iloc[:,:-1]
bivouak_data = data.to_numpy()
target = df_toteach.iloc[:,-1:]
bivouak_target = target.to_numpy().reshape(len(df_toteach))  # Нужно, потому что функция потерь принимает одномерный тензор, а тут без reshape получается массив массивов

In [34]:
data.head(5)   # Проверить. что идет по форме в модель в eval mode 

,r_value,height,inner_tent_height,depth,rope_diameter,sl_bag_length,sl_bag_height,main_axe_load,volume,real_weight,fill_power_1 000,fill_power_550,fill_power_600,fill_power_650,fill_power_700,fill_power_750,fill_power_800,fill_power_850,fill_power_900,vent_back_0,vent_back_Да,vent_back_Нет,vent_zip_0,vent_zip_Без вентиляционных молний,vent_zip_На бёдрах,vent_zip_На бёдрах и голенях,vent_zip_От бедра до низа брючин,vent_zip_От коленей до низа брючин,vent_zip_От талии до низа брючин,vent_zip_Под рукавами,vent_helmet_0,vent_helmet_Активная вентиляция,vent_helmet_Пассивная вентиляция,rope_type_0,rope_type_Динамика,rope_type_Паракорд,rope_type_Репшнур,rope_type_Статика,rope_type_Стропа,sling_type_0,sling_type_Оттяжки с карабинами,sling_type_Петли и оттяжки,warming_type_0,warming_type_Комбинированный,warming_type_Пух,warming_type_Синтетический,warming_type_Хлопок,waterproof_back_0,waterproof_back_Да,waterproof_back_Нет,sleep_bag_person_0,sleep_bag_person_Двухместный,sleep_bag_person_Одноместный,inner_short_0,inner_short_Нет,inner_short_Тайтсы (с компрессией),inner_short_Трусы,inner_short_Шорты (без компрессии),inner_short_Шорты (с компрессией),cloth_collar_0,cloth_collar_V-образный,cloth_collar_Высокий,cloth_collar_Круглый,boot_height_0,boot_height_Высокий,boot_height_Низкий,boot_height_Средний,rub_boot_height_0,rub_boot_height_Высокие,rub_boot_height_Средние,water_resist_0,water_resist_Да,water_resist_Нет,hydro_compl_0,hydro_compl_Да,hydro_compl_Нет,food_ready_0,food_ready_Готовое,food_ready_Залить кипятком,breast_lock_0,breast_lock_Есть,breast_lock_Нет,rope_length_0.0,rope_length_1.0,rope_length_10.0,rope_length_25.0,rope_length_30.0,rope_length_50.0,rope_length_60.0,pants_lenght_0,pants_lenght_Long,pants_lenght_Regular,pants_lenght_Short,pants_lenght_Длинные,pants_lenght_Кальсоны 3/4,pants_lenght_Укороченные,socks_lenght_0,socks_lenght_Высокие,socks_lenght_Гольфы,socks_lenght_Низкие,socks_lenght_Средние,boot_rigid_0,boot_rigid_Высокая жёсткость,boot_rigid_Низкая жёсткость,boot_rigid_Средняя жёсткость,back_pocket_2access_0,back_pocket_2access_Да,back_pocket_2access_Нет,closed_toe_0,closed_toe_Да,closed_toe_Нет,hood_0,hood_Без капюшона,hood_С капюшоном,waist_pocket_0,waist_pocket_Да,waist_pocket_Нет,bottle_pockets_0,bottle_pockets_Да,bottle_pockets_Нет,filter_cat_0,filter_cat_S0,filter_cat_S1,filter_cat_S2,filter_cat_S3,filter_cat_S3+,filter_cat_S4,filter_cat_Фотохром 0-2,filter_cat_Фотохром 0-3,filter_cat_Фотохром 1-2,filter_cat_Фотохром 1-3,filter_cat_Фотохром 2-3,filter_cat_Фотохром 2-4,entry_num_0.0,entry_num_1.0,entry_num_2.0,entry_num_3.0,person_num_0.0,person_num_1.0,person_num_2.0,person_num_3.0,person_num_4.0,person_num_5.0,person_num_6.0,person_num_8.0,person_num_10.0,buckles_num_0,buckles_num_2,buckles_num_3,buckles_num_4 и более,layers_num_0.0,layers_num_2.0,layers_num_2.5,layers_num_3.0,vestibule_num_0,vestibule_num_0.1,vestibule_num_1,vestibule_num_2,vestibule_num_Без тамбура,compression_0,compression_Без компрессии,compression_С компрессией,sandals_const_0,sandals_const_Сабо,sandals_const_Сандалии,boot_const_0,boot_const_Двойные,boot_const_Одинарные,pants_const_0,pants_const_Брюки_классические/,pants_const_Превращаются_в_шорты/,pants_const_Самосбросы/,hammock_const_0,hammock_const_Для снаряжения,hammock_const_Классический,hammock_const_Мостовой,stove_const_0,stove_const_горелки без шланга,stove_const_горелки со шлангом,stove_const_системы приготовления,tent_const_0,tent_const_Для гамака,tent_const_Классический,tent_const_Шатёр,helmet_const_0,helmet_const_Hard Shell,helmet_const_Hybrid,helmet_const_In-Mold,bottle_fast_0,bottle_fast_Без крепления для бутылки,bottle_fast_Гидратор в комплекте,bottle_fast_Карман для бутылки,iceaxe_fast_0,iceaxe_fast_да,iceaxe_fast_нет,poles_fast_0,poles_fast_да,poles_fast_нет,helmet_fast_0,helmet_fast_да,helmet_fast_нет,pants_cut_0,pants_cut_Облегающий,pants_cut_Свободный,pant_boot_hook_0,pant_boot_hook_Есть,pant_boot_hook_Нет,bag_hook_0,bag_hook_Есть,bag_hook_Нет,poles_material_0,poles_m

In [35]:
#data.info(verbose = True, show_counts = True)

In [36]:
bivouak_data.shape

(47818, 478)

In [37]:
bivouak_target.shape

(47818,)

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
X_train, X_test, y_train, y_test = train_test_split(bivouak_data, bivouak_target)

In [40]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_check =  scaler.transform(bivouak_data)    #  !!!!!!! Вот тут!!!! этим же скалером нужно потом новые данные масштабировать????? 

In [41]:
import torch
print('PyTorch version: {}'.format(torch.__version__))
import matplotlib.pyplot as plt
%matplotlib inline
from torch import nn
from IPython.display import clear_output

PyTorch version: 2.2.1


In [42]:
X_train_biv = torch.FloatTensor(X_train)
X_test_biv = torch.FloatTensor(X_test)
X_check_biv = torch.FloatTensor(X_check)

y_train_biv = torch.LongTensor(y_train)
y_test_biv = torch.LongTensor(y_test)

In [43]:
X_train_biv.shape

torch.Size([35863, 478])

In [44]:
y_train_biv.shape

torch.Size([35863])

In [45]:
print(y_train_biv)

tensor([100, 310, 131,  ..., 130, 123, 154])


In [47]:
N_FEATURES = X_train.shape[1]

N_CLASSES = 400
NUM_EPOCH = 500

In [46]:
def create_model(activation, num_features=N_FEATURES, n_out=N_CLASSES):
    model = nn.Sequential()
    model.add_module('lin1', nn.Linear(num_features, 64))
    model.add_module('activation1', activation())
    model.add_module('lin2', nn.Linear(64, 128))
    model.add_module('activation2', activation())
    model.add_module('lin3', nn.Linear(128, 256))
    model.add_module('activation3', activation())
    model.add_module('lin4', nn.Linear(256, 512))
    model.add_module('activation4', activation())
    model.add_module('lin5', nn.Linear(512, 1024))
    model.add_module('activation5', activation())
    model.add_module('lin6', nn.Linear(1024, 512))
    model.add_module('activation6', activation())
    model.add_module('lin7', nn.Linear(512, n_out))

    return model

In [47]:
def train_model(model, X_train_biv, y_train_biv):
    loss_function = nn.CrossEntropyLoss()
    opt = torch.optim.SGD(model.parameters(), lr=0.1)


    loss_history, var_layer_1, var_layer_2, var_layer_3 = [], [], [], []

    for epoch_num in range(NUM_EPOCH):
        opt.zero_grad()

        y_predicted = model(X_train_biv)
        loss = loss_function(y_predicted, y_train_biv)
        reg = 0.005 * (torch.norm(model[0].weight) \
                      + torch.norm(model[2].weight) \
                      + torch.norm(model[4].weight) \
                      + torch.norm(model[6].weight) \
                      + torch.norm(model[8].weight) \
                      + torch.norm(model[10].weight) \
                      + torch.norm(model[12].weight))
        loss += reg
        loss.backward()
        opt.step()

        #loss_history.append(loss.item()) # Always use .item() to store scalars in logs!

       # var_layer_1.append(torch.sum(torch.abs(model[0].weight.grad)).item()/model[0].weight.shape[0])
       # var_layer_2.append(torch.sum(torch.abs(model[2].weight.grad)).item()/model[2].weight.shape[0])
       # var_layer_3.append(torch.sum(torch.abs(model[4].weight.grad)).item()/model[4].weight.shape[0])

    return model
#, loss_history, var_layer_1, var_layer_2, var_layer_3

In [48]:
def create_model_2(activation, num_features=N_FEATURES, n_out=N_CLASSES):
    model = nn.Sequential()
    model.add_module('lin1', nn.Linear(num_features, 1024))
    model.add_module('activation1', activation())
    model.add_module('lin2', nn.Linear(1024, 512))
    model.add_module('activation2', activation())
    model.add_module('lin3', nn.Linear(512, 256))
    model.add_module('activation3', activation())
    model.add_module('lin4', nn.Linear(256, 128))
    model.add_module('activation4', activation())
    model.add_module('lin5', nn.Linear(128, 64))
    model.add_module('activation5', activation())
    model.add_module('lin6', nn.Linear(64, 128))
    model.add_module('activation6', activation())
    model.add_module('lin7', nn.Linear(128, 256))
    model.add_module('activation7', activation())
    model.add_module('lin8', nn.Linear(256, n_out))

    return model

In [49]:
def train_model_2(model, X_train_biv, y_train_biv):
    loss_function = nn.CrossEntropyLoss()
    opt = torch.optim.SGD(model.parameters(), lr=0.1)


    loss_history, var_layer_1, var_layer_2, var_layer_3 = [], [], [], []

    for epoch_num in range(NUM_EPOCH):
        opt.zero_grad()

        y_predicted = model(X_train_biv)
        loss = loss_function(y_predicted, y_train_biv)
        reg = 0.005 * (torch.norm(model[0].weight) \
                      + torch.norm(model[2].weight) \
                      + torch.norm(model[4].weight) \
                      + torch.norm(model[6].weight) \
                      + torch.norm(model[8].weight) \
                      + torch.norm(model[10].weight) \
                      + torch.norm(model[12].weight) \
                      + torch.norm(model[14].weight))
        loss += reg
        loss.backward()
        opt.step()

        #loss_history.append(loss.item()) # Always use .item() to store scalars in logs!

       # var_layer_1.append(torch.sum(torch.abs(model[0].weight.grad)).item()/model[0].weight.shape[0])
       # var_layer_2.append(torch.sum(torch.abs(model[2].weight.grad)).item()/model[2].weight.shape[0])
       # var_layer_3.append(torch.sum(torch.abs(model[4].weight.grad)).item()/model[4].weight.shape[0])

    return model
#, loss_history, var_layer_1, var_layer_2, var_layer_3

In [50]:
model_sigmoid = create_model(nn.Sigmoid)

In [51]:
%%time
out_sigmoid = train_model(model_sigmoid, X_train_biv, y_train_biv)

CPU times: total: 48min 20s
Wall time: 12min 41s


In [52]:
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
train_predictions = torch.max(model_sigmoid(X_train_biv).detach().cpu(), dim=1)[1].numpy()
test_predictions = torch.max(model_sigmoid(X_test_biv).detach().cpu(), dim=1)[1].numpy()

print('train accuracy', accuracy_score(train_predictions, y_train_biv))
print('test accuracy', accuracy_score(test_predictions, y_test_biv))

train accuracy 0.051027521400886705
test accuracy 0.05144291091593475


In [53]:
print(train_predictions)

[175 175 175 ... 175 175 175]


In [54]:
with torch.no_grad():
    Z = model_sigmoid(X_check_biv).numpy()
clust = np.argmax(Z, axis=1)

In [55]:
clust.shape

(47818,)

In [56]:
print(clust)

[175 175 175 ... 175 175 175]


In [57]:
#print(out_sigmoid)

In [58]:
df_tocheck = df_tocheck.copy()
df_tocheck['nn_clusters'] = clust.tolist()

In [59]:
df_tocheck.shape

(47818, 481)

In [60]:
df_tocheck.head(5)

,name,r_value,height,inner_tent_height,depth,rope_diameter,sl_bag_length,sl_bag_height,main_axe_load,volume,real_weight,fill_power_1 000,fill_power_550,fill_power_600,fill_power_650,fill_power_700,fill_power_750,fill_power_800,fill_power_850,fill_power_900,vent_back_0,vent_back_Да,vent_back_Нет,vent_zip_0,vent_zip_Без вентиляционных молний,vent_zip_На бёдрах,vent_zip_На бёдрах и голенях,vent_zip_От бедра до низа брючин,vent_zip_От коленей до низа брючин,vent_zip_От талии до низа брючин,vent_zip_Под рукавами,vent_helmet_0,vent_helmet_Активная вентиляция,vent_helmet_Пассивная вентиляция,rope_type_0,rope_type_Динамика,rope_type_Паракорд,rope_type_Репшнур,rope_type_Статика,rope_type_Стропа,sling_type_0,sling_type_Оттяжки с карабинами,sling_type_Петли и оттяжки,warming_type_0,warming_type_Комбинированный,warming_type_Пух,warming_type_Синтетический,warming_type_Хлопок,waterproof_back_0,waterproof_back_Да,waterproof_back_Нет,sleep_bag_person_0,sleep_bag_person_Двухместный,sleep_bag_person_Одноместный,inner_short_0,inner_short_Нет,inner_short_Тайтсы (с компрессией),inner_short_Трусы,inner_short_Шорты (без компрессии),inner_short_Шорты (с компрессией),cloth_collar_0,cloth_collar_V-образный,cloth_collar_Высокий,cloth_collar_Круглый,boot_height_0,boot_height_Высокий,boot_height_Низкий,boot_height_Средний,rub_boot_height_0,rub_boot_height_Высокие,rub_boot_height_Средние,water_resist_0,water_resist_Да,water_resist_Нет,hydro_compl_0,hydro_compl_Да,hydro_compl_Нет,food_ready_0,food_ready_Готовое,food_ready_Залить кипятком,breast_lock_0,breast_lock_Есть,breast_lock_Нет,rope_length_0.0,rope_length_1.0,rope_length_10.0,rope_length_25.0,rope_length_30.0,rope_length_50.0,rope_length_60.0,pants_lenght_0,pants_lenght_Long,pants_lenght_Regular,pants_lenght_Short,pants_lenght_Длинные,pants_lenght_Кальсоны 3/4,pants_lenght_Укороченные,socks_lenght_0,socks_lenght_Высокие,socks_lenght_Гольфы,socks_lenght_Низкие,socks_lenght_Средние,boot_rigid_0,boot_rigid_Высокая жёсткость,boot_rigid_Низкая жёсткость,boot_rigid_Средняя жёсткость,back_pocket_2access_0,back_pocket_2access_Да,back_pocket_2access_Нет,closed_toe_0,closed_toe_Да,closed_toe_Нет,hood_0,hood_Без капюшона,hood_С капюшоном,waist_pocket_0,waist_pocket_Да,waist_pocket_Нет,bottle_pockets_0,bottle_pockets_Да,bottle_pockets_Нет,filter_cat_0,filter_cat_S0,filter_cat_S1,filter_cat_S2,filter_cat_S3,filter_cat_S3+,filter_cat_S4,filter_cat_Фотохром 0-2,filter_cat_Фотохром 0-3,filter_cat_Фотохром 1-2,filter_cat_Фотохром 1-3,filter_cat_Фотохром 2-3,filter_cat_Фотохром 2-4,entry_num_0.0,entry_num_1.0,entry_num_2.0,entry_num_3.0,person_num_0.0,person_num_1.0,person_num_2.0,person_num_3.0,person_num_4.0,person_num_5.0,person_num_6.0,person_num_8.0,person_num_10.0,buckles_num_0,buckles_num_2,buckles_num_3,buckles_num_4 и более,layers_num_0.0,layers_num_2.0,layers_num_2.5,layers_num_3.0,vestibule_num_0,vestibule_num_0.1,vestibule_num_1,vestibule_num_2,vestibule_num_Без тамбура,compression_0,compression_Без компрессии,compression_С компрессией,sandals_const_0,sandals_const_Сабо,sandals_const_Сандалии,boot_const_0,boot_const_Двойные,boot_const_Одинарные,pants_const_0,pants_const_Брюки_классические/,pants_const_Превращаются_в_шорты/,pants_const_Самосбросы/,hammock_const_0,hammock_const_Для снаряжения,hammock_const_Классический,hammock_const_Мостовой,stove_const_0,stove_const_горелки без шланга,stove_const_горелки со шлангом,stove_const_системы приготовления,tent_const_0,tent_const_Для гамака,tent_const_Классический,tent_const_Шатёр,helmet_const_0,helmet_const_Hard Shell,helmet_const_Hybrid,helmet_const_In-Mold,bottle_fast_0,bottle_fast_Без крепления для бутылки,bottle_fast_Гидратор в комплекте,bottle_fast_Карман для бутылки,iceaxe_fast_0,iceaxe_fast_да,iceaxe_fast_нет,poles_fast_0,poles_fast_да,poles_fast_нет,helmet_fast_0,helmet_fast_да,helmet_fast_нет,pants_cut_0,pants_cut_Облегающий,pants_cut_Свободный,pant_boot_hook_0,pant_boot_hook_Есть,pant_boot_hook_Нет,bag_hook_0,bag_hook_Есть,bag_hook_Нет,poles_material_0,po

In [62]:
df_inside_cluster = df_tocheck.loc[(df_tocheck['nn_clusters'] == 175)]
df_inside_cluster.head(5)

,name,r_value,height,inner_tent_height,depth,rope_diameter,sl_bag_length,sl_bag_height,main_axe_load,volume,real_weight,fill_power_1 000,fill_power_550,fill_power_600,fill_power_650,fill_power_700,fill_power_750,fill_power_800,fill_power_850,fill_power_900,vent_back_0,vent_back_Да,vent_back_Нет,vent_zip_0,vent_zip_Без вентиляционных молний,vent_zip_На бёдрах,vent_zip_На бёдрах и голенях,vent_zip_От бедра до низа брючин,vent_zip_От коленей до низа брючин,vent_zip_От талии до низа брючин,vent_zip_Под рукавами,vent_helmet_0,vent_helmet_Активная вентиляция,vent_helmet_Пассивная вентиляция,rope_type_0,rope_type_Динамика,rope_type_Паракорд,rope_type_Репшнур,rope_type_Статика,rope_type_Стропа,sling_type_0,sling_type_Оттяжки с карабинами,sling_type_Петли и оттяжки,warming_type_0,warming_type_Комбинированный,warming_type_Пух,warming_type_Синтетический,warming_type_Хлопок,waterproof_back_0,waterproof_back_Да,waterproof_back_Нет,sleep_bag_person_0,sleep_bag_person_Двухместный,sleep_bag_person_Одноместный,inner_short_0,inner_short_Нет,inner_short_Тайтсы (с компрессией),inner_short_Трусы,inner_short_Шорты (без компрессии),inner_short_Шорты (с компрессией),cloth_collar_0,cloth_collar_V-образный,cloth_collar_Высокий,cloth_collar_Круглый,boot_height_0,boot_height_Высокий,boot_height_Низкий,boot_height_Средний,rub_boot_height_0,rub_boot_height_Высокие,rub_boot_height_Средние,water_resist_0,water_resist_Да,water_resist_Нет,hydro_compl_0,hydro_compl_Да,hydro_compl_Нет,food_ready_0,food_ready_Готовое,food_ready_Залить кипятком,breast_lock_0,breast_lock_Есть,breast_lock_Нет,rope_length_0.0,rope_length_1.0,rope_length_10.0,rope_length_25.0,rope_length_30.0,rope_length_50.0,rope_length_60.0,pants_lenght_0,pants_lenght_Long,pants_lenght_Regular,pants_lenght_Short,pants_lenght_Длинные,pants_lenght_Кальсоны 3/4,pants_lenght_Укороченные,socks_lenght_0,socks_lenght_Высокие,socks_lenght_Гольфы,socks_lenght_Низкие,socks_lenght_Средние,boot_rigid_0,boot_rigid_Высокая жёсткость,boot_rigid_Низкая жёсткость,boot_rigid_Средняя жёсткость,back_pocket_2access_0,back_pocket_2access_Да,back_pocket_2access_Нет,closed_toe_0,closed_toe_Да,closed_toe_Нет,hood_0,hood_Без капюшона,hood_С капюшоном,waist_pocket_0,waist_pocket_Да,waist_pocket_Нет,bottle_pockets_0,bottle_pockets_Да,bottle_pockets_Нет,filter_cat_0,filter_cat_S0,filter_cat_S1,filter_cat_S2,filter_cat_S3,filter_cat_S3+,filter_cat_S4,filter_cat_Фотохром 0-2,filter_cat_Фотохром 0-3,filter_cat_Фотохром 1-2,filter_cat_Фотохром 1-3,filter_cat_Фотохром 2-3,filter_cat_Фотохром 2-4,entry_num_0.0,entry_num_1.0,entry_num_2.0,entry_num_3.0,person_num_0.0,person_num_1.0,person_num_2.0,person_num_3.0,person_num_4.0,person_num_5.0,person_num_6.0,person_num_8.0,person_num_10.0,buckles_num_0,buckles_num_2,buckles_num_3,buckles_num_4 и более,layers_num_0.0,layers_num_2.0,layers_num_2.5,layers_num_3.0,vestibule_num_0,vestibule_num_0.1,vestibule_num_1,vestibule_num_2,vestibule_num_Без тамбура,compression_0,compression_Без компрессии,compression_С компрессией,sandals_const_0,sandals_const_Сабо,sandals_const_Сандалии,boot_const_0,boot_const_Двойные,boot_const_Одинарные,pants_const_0,pants_const_Брюки_классические/,pants_const_Превращаются_в_шорты/,pants_const_Самосбросы/,hammock_const_0,hammock_const_Для снаряжения,hammock_const_Классический,hammock_const_Мостовой,stove_const_0,stove_const_горелки без шланга,stove_const_горелки со шлангом,stove_const_системы приготовления,tent_const_0,tent_const_Для гамака,tent_const_Классический,tent_const_Шатёр,helmet_const_0,helmet_const_Hard Shell,helmet_const_Hybrid,helmet_const_In-Mold,bottle_fast_0,bottle_fast_Без крепления для бутылки,bottle_fast_Гидратор в комплекте,bottle_fast_Карман для бутылки,iceaxe_fast_0,iceaxe_fast_да,iceaxe_fast_нет,poles_fast_0,poles_fast_да,poles_fast_нет,helmet_fast_0,helmet_fast_да,helmet_fast_нет,pants_cut_0,pants_cut_Облегающий,pants_cut_Свободный,pant_boot_hook_0,pant_boot_hook_Есть,pant_boot_hook_Нет,bag_hook_0,bag_hook_Есть,bag_hook_Нет,poles_material_0,po

In [63]:
df_inside_cluster.shape

(47818, 481)

In [64]:
model_relu = create_model(nn.ReLU)

In [65]:
%%time
out_relu = train_model(model_relu, X_train_biv, y_train_biv)

CPU times: total: 43min 28s
Wall time: 11min 53s


In [66]:
train_predictions_relu = torch.max(model_relu(X_train_biv).detach().cpu(), dim=1)[1].numpy()
test_predictions_relu = torch.max(model_relu(X_test_biv).detach().cpu(), dim=1)[1].numpy()

print('train accuracy', accuracy_score(train_predictions_relu, y_train_biv))
print('test accuracy', accuracy_score(test_predictions_relu, y_test_biv))

train accuracy 0.051027521400886705
test accuracy 0.05144291091593475


In [67]:
print(train_predictions_relu)

[175 175 175 ... 175 175 175]


In [68]:
with torch.no_grad():
    Zr = model_relu(X_check_biv).numpy()
clust_relu = np.argmax(Zr, axis=1)

In [69]:
clust_relu.shape

(47818,)

In [70]:
print(clust_relu)

[175 175 175 ... 175 175 175]


In [71]:
df_tocheck = df_tocheck.copy()
df_tocheck['nn_clusters_relu'] = clust_relu.tolist()

In [72]:
df_tocheck.shape

(47818, 482)

In [73]:
df_tocheck.head(5)

,name,r_value,height,inner_tent_height,depth,rope_diameter,sl_bag_length,sl_bag_height,main_axe_load,volume,real_weight,fill_power_1 000,fill_power_550,fill_power_600,fill_power_650,fill_power_700,fill_power_750,fill_power_800,fill_power_850,fill_power_900,vent_back_0,vent_back_Да,vent_back_Нет,vent_zip_0,vent_zip_Без вентиляционных молний,vent_zip_На бёдрах,vent_zip_На бёдрах и голенях,vent_zip_От бедра до низа брючин,vent_zip_От коленей до низа брючин,vent_zip_От талии до низа брючин,vent_zip_Под рукавами,vent_helmet_0,vent_helmet_Активная вентиляция,vent_helmet_Пассивная вентиляция,rope_type_0,rope_type_Динамика,rope_type_Паракорд,rope_type_Репшнур,rope_type_Статика,rope_type_Стропа,sling_type_0,sling_type_Оттяжки с карабинами,sling_type_Петли и оттяжки,warming_type_0,warming_type_Комбинированный,warming_type_Пух,warming_type_Синтетический,warming_type_Хлопок,waterproof_back_0,waterproof_back_Да,waterproof_back_Нет,sleep_bag_person_0,sleep_bag_person_Двухместный,sleep_bag_person_Одноместный,inner_short_0,inner_short_Нет,inner_short_Тайтсы (с компрессией),inner_short_Трусы,inner_short_Шорты (без компрессии),inner_short_Шорты (с компрессией),cloth_collar_0,cloth_collar_V-образный,cloth_collar_Высокий,cloth_collar_Круглый,boot_height_0,boot_height_Высокий,boot_height_Низкий,boot_height_Средний,rub_boot_height_0,rub_boot_height_Высокие,rub_boot_height_Средние,water_resist_0,water_resist_Да,water_resist_Нет,hydro_compl_0,hydro_compl_Да,hydro_compl_Нет,food_ready_0,food_ready_Готовое,food_ready_Залить кипятком,breast_lock_0,breast_lock_Есть,breast_lock_Нет,rope_length_0.0,rope_length_1.0,rope_length_10.0,rope_length_25.0,rope_length_30.0,rope_length_50.0,rope_length_60.0,pants_lenght_0,pants_lenght_Long,pants_lenght_Regular,pants_lenght_Short,pants_lenght_Длинные,pants_lenght_Кальсоны 3/4,pants_lenght_Укороченные,socks_lenght_0,socks_lenght_Высокие,socks_lenght_Гольфы,socks_lenght_Низкие,socks_lenght_Средние,boot_rigid_0,boot_rigid_Высокая жёсткость,boot_rigid_Низкая жёсткость,boot_rigid_Средняя жёсткость,back_pocket_2access_0,back_pocket_2access_Да,back_pocket_2access_Нет,closed_toe_0,closed_toe_Да,closed_toe_Нет,hood_0,hood_Без капюшона,hood_С капюшоном,waist_pocket_0,waist_pocket_Да,waist_pocket_Нет,bottle_pockets_0,bottle_pockets_Да,bottle_pockets_Нет,filter_cat_0,filter_cat_S0,filter_cat_S1,filter_cat_S2,filter_cat_S3,filter_cat_S3+,filter_cat_S4,filter_cat_Фотохром 0-2,filter_cat_Фотохром 0-3,filter_cat_Фотохром 1-2,filter_cat_Фотохром 1-3,filter_cat_Фотохром 2-3,filter_cat_Фотохром 2-4,entry_num_0.0,entry_num_1.0,entry_num_2.0,entry_num_3.0,person_num_0.0,person_num_1.0,person_num_2.0,person_num_3.0,person_num_4.0,person_num_5.0,person_num_6.0,person_num_8.0,person_num_10.0,buckles_num_0,buckles_num_2,buckles_num_3,buckles_num_4 и более,layers_num_0.0,layers_num_2.0,layers_num_2.5,layers_num_3.0,vestibule_num_0,vestibule_num_0.1,vestibule_num_1,vestibule_num_2,vestibule_num_Без тамбура,compression_0,compression_Без компрессии,compression_С компрессией,sandals_const_0,sandals_const_Сабо,sandals_const_Сандалии,boot_const_0,boot_const_Двойные,boot_const_Одинарные,pants_const_0,pants_const_Брюки_классические/,pants_const_Превращаются_в_шорты/,pants_const_Самосбросы/,hammock_const_0,hammock_const_Для снаряжения,hammock_const_Классический,hammock_const_Мостовой,stove_const_0,stove_const_горелки без шланга,stove_const_горелки со шлангом,stove_const_системы приготовления,tent_const_0,tent_const_Для гамака,tent_const_Классический,tent_const_Шатёр,helmet_const_0,helmet_const_Hard Shell,helmet_const_Hybrid,helmet_const_In-Mold,bottle_fast_0,bottle_fast_Без крепления для бутылки,bottle_fast_Гидратор в комплекте,bottle_fast_Карман для бутылки,iceaxe_fast_0,iceaxe_fast_да,iceaxe_fast_нет,poles_fast_0,poles_fast_да,poles_fast_нет,helmet_fast_0,helmet_fast_да,helmet_fast_нет,pants_cut_0,pants_cut_Облегающий,pants_cut_Свободный,pant_boot_hook_0,pant_boot_hook_Есть,pant_boot_hook_Нет,bag_hook_0,bag_hook_Есть,bag_hook_Нет,poles_material_0,po

In [74]:
df_inside_cluster = df_tocheck.loc[(df_tocheck['nn_clusters_relu'] == 175)]
df_inside_cluster.head(20)

,name,r_value,height,inner_tent_height,depth,rope_diameter,sl_bag_length,sl_bag_height,main_axe_load,volume,real_weight,fill_power_1 000,fill_power_550,fill_power_600,fill_power_650,fill_power_700,fill_power_750,fill_power_800,fill_power_850,fill_power_900,vent_back_0,vent_back_Да,vent_back_Нет,vent_zip_0,vent_zip_Без вентиляционных молний,vent_zip_На бёдрах,vent_zip_На бёдрах и голенях,vent_zip_От бедра до низа брючин,vent_zip_От коленей до низа брючин,vent_zip_От талии до низа брючин,vent_zip_Под рукавами,vent_helmet_0,vent_helmet_Активная вентиляция,vent_helmet_Пассивная вентиляция,rope_type_0,rope_type_Динамика,rope_type_Паракорд,rope_type_Репшнур,rope_type_Статика,rope_type_Стропа,sling_type_0,sling_type_Оттяжки с карабинами,sling_type_Петли и оттяжки,warming_type_0,warming_type_Комбинированный,warming_type_Пух,warming_type_Синтетический,warming_type_Хлопок,waterproof_back_0,waterproof_back_Да,waterproof_back_Нет,sleep_bag_person_0,sleep_bag_person_Двухместный,sleep_bag_person_Одноместный,inner_short_0,inner_short_Нет,inner_short_Тайтсы (с компрессией),inner_short_Трусы,inner_short_Шорты (без компрессии),inner_short_Шорты (с компрессией),cloth_collar_0,cloth_collar_V-образный,cloth_collar_Высокий,cloth_collar_Круглый,boot_height_0,boot_height_Высокий,boot_height_Низкий,boot_height_Средний,rub_boot_height_0,rub_boot_height_Высокие,rub_boot_height_Средние,water_resist_0,water_resist_Да,water_resist_Нет,hydro_compl_0,hydro_compl_Да,hydro_compl_Нет,food_ready_0,food_ready_Готовое,food_ready_Залить кипятком,breast_lock_0,breast_lock_Есть,breast_lock_Нет,rope_length_0.0,rope_length_1.0,rope_length_10.0,rope_length_25.0,rope_length_30.0,rope_length_50.0,rope_length_60.0,pants_lenght_0,pants_lenght_Long,pants_lenght_Regular,pants_lenght_Short,pants_lenght_Длинные,pants_lenght_Кальсоны 3/4,pants_lenght_Укороченные,socks_lenght_0,socks_lenght_Высокие,socks_lenght_Гольфы,socks_lenght_Низкие,socks_lenght_Средние,boot_rigid_0,boot_rigid_Высокая жёсткость,boot_rigid_Низкая жёсткость,boot_rigid_Средняя жёсткость,back_pocket_2access_0,back_pocket_2access_Да,back_pocket_2access_Нет,closed_toe_0,closed_toe_Да,closed_toe_Нет,hood_0,hood_Без капюшона,hood_С капюшоном,waist_pocket_0,waist_pocket_Да,waist_pocket_Нет,bottle_pockets_0,bottle_pockets_Да,bottle_pockets_Нет,filter_cat_0,filter_cat_S0,filter_cat_S1,filter_cat_S2,filter_cat_S3,filter_cat_S3+,filter_cat_S4,filter_cat_Фотохром 0-2,filter_cat_Фотохром 0-3,filter_cat_Фотохром 1-2,filter_cat_Фотохром 1-3,filter_cat_Фотохром 2-3,filter_cat_Фотохром 2-4,entry_num_0.0,entry_num_1.0,entry_num_2.0,entry_num_3.0,person_num_0.0,person_num_1.0,person_num_2.0,person_num_3.0,person_num_4.0,person_num_5.0,person_num_6.0,person_num_8.0,person_num_10.0,buckles_num_0,buckles_num_2,buckles_num_3,buckles_num_4 и более,layers_num_0.0,layers_num_2.0,layers_num_2.5,layers_num_3.0,vestibule_num_0,vestibule_num_0.1,vestibule_num_1,vestibule_num_2,vestibule_num_Без тамбура,compression_0,compression_Без компрессии,compression_С компрессией,sandals_const_0,sandals_const_Сабо,sandals_const_Сандалии,boot_const_0,boot_const_Двойные,boot_const_Одинарные,pants_const_0,pants_const_Брюки_классические/,pants_const_Превращаются_в_шорты/,pants_const_Самосбросы/,hammock_const_0,hammock_const_Для снаряжения,hammock_const_Классический,hammock_const_Мостовой,stove_const_0,stove_const_горелки без шланга,stove_const_горелки со шлангом,stove_const_системы приготовления,tent_const_0,tent_const_Для гамака,tent_const_Классический,tent_const_Шатёр,helmet_const_0,helmet_const_Hard Shell,helmet_const_Hybrid,helmet_const_In-Mold,bottle_fast_0,bottle_fast_Без крепления для бутылки,bottle_fast_Гидратор в комплекте,bottle_fast_Карман для бутылки,iceaxe_fast_0,iceaxe_fast_да,iceaxe_fast_нет,poles_fast_0,poles_fast_да,poles_fast_нет,helmet_fast_0,helmet_fast_да,helmet_fast_нет,pants_cut_0,pants_cut_Облегающий,pants_cut_Свободный,pant_boot_hook_0,pant_boot_hook_Есть,pant_boot_hook_Нет,bag_hook_0,bag_hook_Есть,bag_hook_Нет,poles_material_0,po

In [75]:
df_inside_cluster.shape

(47818, 482)

In [76]:
model_sigmoid_2 = create_model_2(nn.Sigmoid)

In [77]:
%%time
out_sigmoid_2 = train_model_2(model_sigmoid_2, X_train_biv, y_train_biv)

CPU times: total: 39min 23s
Wall time: 11min 5s


In [78]:
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
train_predictions_2 = torch.max(model_sigmoid_2(X_train_biv).detach().cpu(), dim=1)[1].numpy()
test_predictions_2 = torch.max(model_sigmoid_2(X_test_biv).detach().cpu(), dim=1)[1].numpy()

print('train accuracy', accuracy_score(train_predictions_2, y_train_biv))
print('test accuracy', accuracy_score(test_predictions_2, y_test_biv))

train accuracy 0.051027521400886705
test accuracy 0.05144291091593475


In [79]:
print(train_predictions_2)

[175 175 175 ... 175 175 175]


In [80]:
model_relu_2 = create_model_2(nn.ReLU)

In [81]:
%%time
out_relu_2 = train_model_2(model_relu_2, X_train_biv, y_train_biv)

CPU times: total: 36min 51s
Wall time: 10min 34s


In [82]:
train_predictions_relu_2 = torch.max(model_relu_2(X_train_biv).detach().cpu(), dim=1)[1].numpy()
test_predictions_relu_2 = torch.max(model_relu_2(X_test_biv).detach().cpu(), dim=1)[1].numpy()

print('train accuracy', accuracy_score(train_predictions_relu_2, y_train_biv))
print('test accuracy', accuracy_score(test_predictions_relu_2, y_test_biv))

train accuracy 0.051027521400886705
test accuracy 0.05144291091593475


In [83]:
print(train_predictions_relu_2)

[175 175 175 ... 175 175 175]


In [46]:
def create_model_3(activation, num_features=N_FEATURES, n_out=N_CLASSES):
    model = nn.Sequential()
    model.add_module('lin1', nn.Linear(num_features, 256))
    model.add_module('activation1', activation())
    model.add_module('lin2', nn.Linear(256, 512))
    model.add_module('activation2', activation())
    model.add_module('lin3', nn.Linear(512, 1024))
    model.add_module('activation3', activation())
    model.add_module('lin4', nn.Linear(1024, 512))
    model.add_module('activation4', activation())
    model.add_module('lin5', nn.Linear(512, 256))
    model.add_module('activation5', activation())
    model.add_module('lin6', nn.Linear(256, n_out))

    return model

In [47]:
def train_model_3(model, X_train_biv, y_train_biv):
    loss_function = nn.CrossEntropyLoss()
    opt = torch.optim.SGD(model.parameters(), lr=0.1)


    loss_history, var_layer_1, var_layer_2, var_layer_3 = [], [], [], []

    for epoch_num in range(NUM_EPOCH):
        opt.zero_grad()

        y_predicted = model(X_train_biv)
        loss = loss_function(y_predicted, y_train_biv)
        reg = 0.005 * (torch.norm(model[0].weight) \
                      + torch.norm(model[2].weight) \
                      + torch.norm(model[4].weight) \
                      + torch.norm(model[6].weight) \
                      + torch.norm(model[8].weight) \
                      + torch.norm(model[10].weight))
        loss += reg
        loss.backward()
        opt.step()

        #loss_history.append(loss.item()) # Always use .item() to store scalars in logs!

       # var_layer_1.append(torch.sum(torch.abs(model[0].weight.grad)).item()/model[0].weight.shape[0])
       # var_layer_2.append(torch.sum(torch.abs(model[2].weight.grad)).item()/model[2].weight.shape[0])
       # var_layer_3.append(torch.sum(torch.abs(model[4].weight.grad)).item()/model[4].weight.shape[0])

    return model
#, loss_history, var_layer_1, var_layer_2, var_layer_3

In [48]:
model_relu_3 = create_model_3(nn.ReLU)

In [49]:
%%time
out_relu_3 = train_model_3(model_relu_3, X_train_biv, y_train_biv)

CPU times: total: 44min 22s
Wall time: 12min 42s


In [51]:
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
train_predictions_relu_3 = torch.max(model_relu_3(X_train_biv).detach().cpu(), dim=1)[1].numpy()
test_predictions_relu_3 = torch.max(model_relu_3(X_test_biv).detach().cpu(), dim=1)[1].numpy()

print('train accuracy', accuracy_score(train_predictions_relu_3, y_train_biv))
print('test accuracy', accuracy_score(test_predictions_relu_3, y_test_biv))

train accuracy 0.0431363801132086
test accuracy 0.04123797574236721


In [52]:
print(train_predictions_relu_3)

[312 312 312 ... 312  79  79]


In [48]:
def create_model_4(activation, num_features=N_FEATURES, n_out=N_CLASSES):
    model = nn.Sequential()
    model.add_module('lin1', nn.Linear(num_features, 1024))  
    model.add_module('lin2', nn.Linear(1024, 512))   
    model.add_module('lin3', nn.Linear(512, 256))    
    model.add_module('lin4', nn.Linear(256, 128))
    model.add_module('activation1', activation())
    model.add_module('lin5', nn.Linear(128, 64))    
    model.add_module('lin6', nn.Linear(64, 128))   
    model.add_module('lin7', nn.Linear(128, 256))
    model.add_module('activation2', activation())
    model.add_module('lin8', nn.Linear(256, n_out))

    return model

In [50]:
def train_model_4(model, X_train_biv, y_train_biv):
    loss_function = nn.CrossEntropyLoss()
    opt = torch.optim.SGD(model.parameters(), lr=0.1)


   

    for epoch_num in range(NUM_EPOCH):
        opt.zero_grad()

        y_predicted = model(X_train_biv)
        loss = loss_function(y_predicted, y_train_biv)
        
        loss.backward()
        opt.step()
    return model


In [51]:
model_relu_4 = create_model_4(nn.ReLU)

In [52]:
%%time
out_relu_4 = train_model_4(model_relu_4, X_train_biv, y_train_biv)

CPU times: total: 43min 6s
Wall time: 9min 42s


In [53]:
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
train_predictions_relu_4 = torch.max(model_relu_4(X_train_biv).detach().cpu(), dim=1)[1].numpy()
test_predictions_relu_4 = torch.max(model_relu_4(X_test_biv).detach().cpu(), dim=1)[1].numpy()

print('train accuracy', accuracy_score(train_predictions_relu_4, y_train_biv))
print('test accuracy', accuracy_score(test_predictions_relu_4, y_test_biv))

train accuracy 0.05735716476591473
test accuracy 0.05872020075282309


In [55]:
print(train_predictions_relu_4)

[ 15  15 290 ... 290 312  15]


In [56]:
with torch.no_grad():
    Zr = model_relu_4(X_check_biv).numpy()
clust_relu4 = np.argmax(Zr, axis=1)

In [57]:
clust_relu4.shape

(47818,)

In [58]:
print(clust_relu4)

[ 79  79  79 ... 312 312 312]


In [59]:
df_tocheck4 = df_tocheck.copy()
df_tocheck4['nn_clusters_relu'] = clust_relu4.tolist()

In [60]:
df_tocheck4.nn_clusters_relu.value_counts()

312    16778
79     10998
15      9338
290     7522
1       2825
58       318
116       39
Name: nn_clusters_relu, dtype: int64

In [80]:
df_pr = pd.read_csv('corr_sales_biv_preprocess_24_test.csv', encoding='cp1251', sep=',', low_memory=False)
pd.set_option('display.max_columns', None)
df_pr.head(5)
df_pr.drop('Unnamed: 0', axis= 1 , inplace= True )

In [81]:
df2_full_pr = df_pr.iloc[:,[1,16,27,28,32,33,34,35,40,43,44]]
df2_data_pr = df_pr.iloc[:,[16,27,28,32,33,34,35,40,43,44]]
df2_data_pr.head(5)

,r_value,height,inner_tent_height,depth,rope_diameter,sl_bag_length,sl_bag_height,main_axe_load,volume,real_weight
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.070
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.070
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.070


In [82]:
df3_pr = df_pr.iloc[:,55:529]
df3_pr.head(5)

,fill_power_1 000,fill_power_550,fill_power_600,fill_power_650,fill_power_700,fill_power_750,fill_power_800,fill_power_850,fill_power_900,vent_back_0,vent_back_Да,vent_back_Нет,vent_zip_0,vent_zip_Без вентиляционных молний,vent_zip_На бёдрах,vent_zip_На бёдрах и голенях,vent_zip_От бедра до низа брючин,vent_zip_От коленей до низа брючин,vent_zip_От талии до низа брючин,vent_zip_Под рукавами,vent_helmet_0,vent_helmet_Активная вентиляция,vent_helmet_Пассивная вентиляция,rope_type_0,rope_type_Динамика,rope_type_Паракорд,rope_type_Репшнур,rope_type_Статика,rope_type_Стропа,sling_type_0,sling_type_Оттяжки с карабинами,sling_type_Петли и оттяжки,warming_type_0,warming_type_Комбинированный,warming_type_Пух,warming_type_Синтетический,warming_type_Хлопок,waterproof_back_0,waterproof_back_Да,waterproof_back_Нет,sleep_bag_person_0,sleep_bag_person_Двухместный,sleep_bag_person_Одноместный,inner_short_0,inner_short_Нет,inner_short_Тайтсы (с компрессией),inner_short_Трусы,inner_short_Шорты (без компрессии),inner_short_Шорты (с компрессией),cloth_collar_0,cloth_collar_V-образный,cloth_collar_Высокий,cloth_collar_Круглый,boot_height_0,boot_height_Высокий,boot_height_Низкий,boot_height_Средний,rub_boot_height_0,rub_boot_height_Высокие,rub_boot_height_Средние,water_resist_0,water_resist_Да,water_resist_Нет,hydro_compl_0,hydro_compl_Да,hydro_compl_Нет,food_ready_0,food_ready_Готовое,food_ready_Залить кипятком,breast_lock_0,breast_lock_Есть,breast_lock_Нет,rope_length_0.0,rope_length_1.0,rope_length_10.0,rope_length_25.0,rope_length_30.0,rope_length_50.0,rope_length_60.0,pants_lenght_0,pants_lenght_Long,pants_lenght_Regular,pants_lenght_Short,pants_lenght_Длинные,pants_lenght_Кальсоны 3/4,pants_lenght_Укороченные,socks_lenght_0,socks_lenght_Высокие,socks_lenght_Гольфы,socks_lenght_Низкие,socks_lenght_Средние,boot_rigid_0,boot_rigid_Высокая жёсткость,boot_rigid_Низкая жёсткость,boot_rigid_Средняя жёсткость,back_pocket_2access_0,back_pocket_2access_Да,back_pocket_2access_Нет,closed_toe_0,closed_toe_Да,closed_toe_Нет,hood_0,hood_Без капюшона,hood_С капюшоном,waist_pocket_0,waist_pocket_Да,waist_pocket_Нет,bottle_pockets_0,bottle_pockets_Да,bottle_pockets_Нет,filter_cat_0,filter_cat_S0,filter_cat_S1,filter_cat_S2,filter_cat_S3,filter_cat_S3+,filter_cat_S4,filter_cat_Фотохром 0-2,filter_cat_Фотохром 0-3,filter_cat_Фотохром 1-2,filter_cat_Фотохром 1-3,filter_cat_Фотохром 2-3,filter_cat_Фотохром 2-4,entry_num_0.0,entry_num_1.0,entry_num_2.0,entry_num_3.0,person_num_0.0,person_num_1.0,person_num_2.0,person_num_3.0,person_num_4.0,person_num_5.0,person_num_6.0,person_num_8.0,person_num_10.0,buckles_num_0,buckles_num_2,buckles_num_3,buckles_num_4 и более,layers_num_0.0,layers_num_2.0,layers_num_2.5,layers_num_3.0,vestibule_num_0,vestibule_num_0.1,vestibule_num_1,vestibule_num_2,vestibule_num_Без тамбура,compression_0,compression_Без компрессии,compression_С компрессией,sandals_const_0,sandals_const_Сабо,sandals_const_Сандалии,boot_const_0,boot_const_Двойные,boot_const_Одинарные,pants_const_0,pants_const_Брюки_классические/,pants_const_Превращаются_в_шорты/,pants_const_Самосбросы/,hammock_const_0,hammock_const_Для снаряжения,hammock_const_Классический,hammock_const_Мостовой,stove_const_0,stove_const_горелки без шланга,stove_const_горелки со шлангом,stove_const_системы приготовления,tent_const_0,tent_const_Для гамака,tent_const_Классический,tent_const_Шатёр,helmet_const_0,helmet_const_Hard Shell,helmet_const_Hybrid,helmet_const_In-Mold,bottle_fast_0,bottle_fast_Без крепления для бутылки,bottle_fast_Гидратор в комплекте,bottle_fast_Карман для бутылки,iceaxe_fast_0,iceaxe_fast_да,iceaxe_fast_нет,poles_fast_0,poles_fast_да,poles_fast_нет,helmet_fast_0,helmet_fast_да,helmet_fast_нет,pants_cut_0,pants_cut_Облегающий,pants_cut_Свободный,pant_boot_hook_0,pant_boot_hook_Есть,pant_boot_hook_Нет,bag_hook_0,bag_hook_Есть,bag_hook_Нет,poles_material_0,poles_material_Алюминий,poles_material_Карбон,poles_material_Композит,poles_material_Сталь,carabiner_material_0,carabiner

In [86]:
df_toteach_pr = pd.concat([df2_data_pr,df3_pr], axis=1)
df_tocheck_pr = pd.concat([df2_full_pr,df3_pr], axis=1)

In [ ]:
#uid_dc_labels = clf.predict(X)

In [ ]:
#df_toteach['clusters'] = uid_dc_labels

In [ ]:
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()

In [ ]:
#le.fit(df_toteach['clusters'])
#clusters_kne = le.transform(df_toteach['clusters'])
#df_toteach = df_toteach.copy()
#df_toteach['clusters'] = clusters_kne

In [ ]:
#data = df_toteach.iloc[:,:-1]
#bivouak_data = data.to_numpy()

In [87]:
bivouak_data_hebd = df_toteach_pr.to_numpy()

In [89]:
bivouak_data_hebd.shape

(7854, 484)

In [90]:
# тут оч большой вопрос, используем ли тот же scalerдля масштабирования????

In [91]:
X_hebd =  scaler.transform(bivouak_data_hebd)

In [92]:
X_hebd_biv = torch.FloatTensor(X_hebd)

In [96]:
with torch.no_grad():
    Zhebd = model_relu(X_hebd_biv).numpy()
clust_hebd = np.argmax(Zhebd, axis=1)

In [97]:
clust_hebd.shape

(7854,)

In [98]:
print(clust_hebd)

[312 312 312 ... 312 312 312]


In [ ]:
#df_pr

In [ ]:
#df_tocheck = df_tocheck.copy()
#df_tocheck['nn_clusters_relu'] = clust_hebd.tolist()

In [ ]:
#df_tocheck.shape

In [ ]:
#df_tocheck.head(5)

In [99]:
df_outcluster_hebd = df_pr.iloc[:,0:6]
#df_outcluster.head(5)

In [100]:
df_outcluster_hebd['nn_clusters_relu'] = clust_hebd.tolist()

In [101]:
df_outcluster_hebd.head(5)

,uid,name,category1_name,category2_name,category3_name,category4_name,nn_clusters_relu
0,00fae3ac-1bab-11e7-80c7-901b0e95a2a8,Ложка-Вилка MSR Blue,Туристическая посуда,Вилки и ложки,NaN,NaN,312
1,00fae3ac-1bab-11e7-80c7-901b0e95a2a8,Ложка-Вилка MSR Blue,Туристическая посуда,Вилки и ложки,NaN,NaN,312
2,05e85429-0f4c-11ee-85da-b42e99c9b482,"Фляга Source Nomadic 1,0 л Blue",Туристическая посуда,"Фляги, канистры, ёмкости",NaN,NaN,312
3,05e85429-0f4c-11ee-85da-b42e99c9b482,"Фляга Source Nomadic 1,0 л Blue",Туристическая посуда,"Фляги, канистры, ёмкости",NaN,NaN,312
4,05e85429-0f4c-11ee-85da-b42e99c9b482,"Фляга Source Nomadic 1,0 л Blue",Туристическая посуда,"Фляги, канистры, ёмкости",NaN,NaN,312


In [102]:
df_outcluster_hebd.to_csv('biv_clustered.csv', encoding='cp1251', sep=',')

Альтернативный трек: запишем выход из KNN

In [61]:
df_pr = pd.read_csv('corr_sales_biv_preprocess_24_test.csv', encoding='cp1251', sep=',', low_memory=False)
pd.set_option('display.max_columns', None)
df_pr.head(5)
df_pr.drop('Unnamed: 0', axis= 1 , inplace= True )

In [64]:
df_outcluster_knn = df.iloc[:,0:6]

In [65]:
df_outcluster_knn['clusters'] = clusters_kne

In [66]:
df_outcluster_knn.head(5)

,uid,name,category1_name,category2_name,category3_name,category4_name,clusters
0,00e7d508-7577-11e9-80ca-901b0e95a2a8,Тент ENO ProFly XL Sil Lichen,Палатки,Тенты и противомоскитные пологи,NaN,NaN,95
1,00e7d508-7577-11e9-80ca-901b0e95a2a8,Тент ENO ProFly XL Sil Lichen,Палатки,Тенты и противомоскитные пологи,NaN,NaN,95
2,00e7d508-7577-11e9-80ca-901b0e95a2a8,Тент ENO ProFly XL Sil Lichen,Палатки,Тенты и противомоскитные пологи,NaN,NaN,95
3,00e7d508-7577-11e9-80ca-901b0e95a2a8,Тент ENO ProFly XL Sil Lichen,Палатки,Тенты и противомоскитные пологи,NaN,NaN,95
4,00e7d508-7577-11e9-80ca-901b0e95a2a8,Тент ENO ProFly XL Sil Lichen,Палатки,Тенты и противомоскитные пологи,NaN,NaN,71


In [67]:
df_outcluster_knn.to_csv('biv_clustered_knn.csv', encoding='cp1251', sep=',')